In [5]:
import tenseal.sealapi as seal
import util
from math import log2, ceil

In [6]:
parms = seal.EncryptionParameters(seal.SCHEME_TYPE.CKKS)
poly_modulus_degree = 2**13

bit_size = 50

mod_chain = [bit_size, 60]

parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(seal.CoeffModulus.Create(poly_modulus_degree, mod_chain))
scale = pow(2, 46)

context = seal.SEALContext(parms, True, seal.SEC_LEVEL_TYPE.TC128)
util.print_parameters(context)


/
|Encryption parameters: 
|	scheme: CKKS
|	poly_modulus_degree: 8192
|	coeff_modulus_size: 110 (50 60) bits
|	Max Bit Count: 218


In [7]:
keygen = seal.KeyGenerator(context)
secret_key = keygen.secret_key()

public_key = seal.PublicKey()
keygen.create_public_key(public_key)

encryptor = seal.Encryptor(context, public_key)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, secret_key)

encoder = seal.CKKSEncoder(context)

In [8]:
x = 3
x_plain = seal.Plaintext()
encoder.encode(x, scale, x_plain)

# This is the check for overflow that is done in Microsoft SEAL. 
coeff_bit_count = log2(abs(x*scale))+2
coeff_modulus_bit_count = context.get_context_data(x_plain.parms_id()).total_coeff_modulus_bit_count()
print(f"coeff_bit_count: {coeff_bit_count}", end = " ")
print(f"total_coeff_modulus_bit_count: {coeff_modulus_bit_count}")
print(f"coeff_bit_count >= total_coeff_modulus_bit_count: {coeff_bit_count>=coeff_modulus_bit_count}")

x_enc = seal.Ciphertext()
encryptor.encrypt(x_plain, x_enc)

coeff_bit_count: 49.584962500721154 total_coeff_modulus_bit_count: 50
coeff_bit_count >= total_coeff_modulus_bit_count: False


In [9]:
result = seal.Plaintext()

decryptor.decrypt(x_enc, result)
after_enc = encoder.decode_double(result)[0]
error = abs(x-after_enc)
print(f"With log2(scale):{log2(scale)} and bit_size:{bit_size}, the")
print(f"\tdecrypted value is {after_enc}")
if error>0:
    print(f"\tabsolute error was: {error}")
else:
    print("\terror is 0")

With log2(scale):46.0 and bit_size:50, the
	decrypted value is 2.999999999985655
	absolute error was: 1.4344969656576723e-11
